# CH3. 실전 프로그램 개발 - BLOG 앱

## 1. 애플리케이션 설계

: 포스트에 대한 리스트를 보여주고, 특정 포스트를 클릭하면 해당 글을 읽을 수 있는 기능을 개발

> **화면 UI 설계**   
> post_list.html : Post.title / Post.modify_date / Post.description / 페이지 이동   
> post_detail.html : Post.title / Post.modify_date / Post.content

> **테이블 설계**   
> Post 테이블   
> id : Integer / PK, AutoIncrement   
> title : CharField(50)   
> slug : SlugField(50), Unique   
> description : CharField(100), Blank   
> content : TextField   
> create_dt(생성) : DateTimeField, auto_now_add  
> modify_dt(수정) : DateTimeField, auto_now

> **로직 & URL 설계**   
> /blog/ - PostLV(ListView) - post_all.html   
> /blog/post/ - PostLV(ListView) - post_all.html   
> /blog/post/django-example/ - PostDV(DetailView) - post_detail.html   
> /blog/archive/ - PostAV(ArchiveIndexView) - post_archieve.html   
> /blog/archive/2019 - PostYAV(YearArchiveView) - post_archieve_year.html   
> /blog/archive/2019/nov/ - PostMAV(MonthArchiveView) - post_archive_month.html   
> /blog/archive/2019/nov/10/ - PostDAV(DayArchiveView) - post_archive_day.html   
> /blog/archive/today/ - PostTAV(TodayArhciveView) - post_archive_day.html   
> /admin/

> **코딩 순서**
> 1. startproject   
> 2. settings.py   
> 3. migrate   
> 4. createsuperuser   
> 5. startapp : Blog 앱 생성   
> 6. settings.py : Blog 앱 등록   
> 7. models.py : 모델 테이블 정의   
> 8. admin.py : 모델 테이블 등록   
> 8. makemigrations / migrate   
> 9. views.py
> 10. templates

## 2. 뼈대 만들기

- Blog 앱 생성   

```
python manage.py startapp blog
```

- Blog 앱 등록   

```
mysite/settings.py

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'bookmark.apps.BookmarkConfig',
    'blog.apps.BlogConfig',
]
```

## 3. 모델 코딩하기

- 테이블 생성

```
blog/models.py

from django.db import models

class Post(models.Model) :
    # id 자동 생성
    title = models.CharField(verbose_name = 'TITLE', max_length = 50)
    # allow_unicode로 한글 처리
    slug = models.SlugField('SLUG', unique = True, allow_unicode = True, help_text = 'one word for title alias.')
    description = models.CharField('DESCRIPTION', max_length = 100, blank = True, help_text = 'simple description text')
    content = models.TextField('CONTENT')
    # 생성 시각 저장 (auto_now_add)
    create_dt = models.DateTimeField('CREATE DATE', auto_now_add = True)
    # 저장 시각 저장 (auto_now)
    modify_dt = models.DateTimeField('MODIFY DATE', auto_now = True)

    # 그 외 필요 파라미터 지정
    class Meta :
        # 테이블의 단수 별칭
        verbose_name = 'post'
        # 테이블의 복수 별칭
        verbose_name_plural = 'posts'
        # DB에 저장되는 테이블명 지정 (기본은 blog_post)
        db_table = 'blog_posts'
        # 수정 기각을 내림차순으로 저장
        ordering = ('-modify_dt',)

    def __str__(self):
        return self.title

    def get_absolute_url(self):
        return reverse('blog:post_detail', args = (self.slug,))

    def get_previous(self):
        return self.get_previous_by_modify_dt()

    def get_next(self):
        return self.get_next_by_modify_dt()
```

- 테이블 등록   

```
blog/admin.py

from django.contrib import admin
from blog.models import Blog

@admin.register(Blog)
# Admin 사이트에서 보여지는 모습 지정
class PostAdmin(admin.ModelAdmin) :
    # 보여지는 컬럼
    list_display = ('id', 'title', 'modify_dt')
    # 필터 사이드바 지정
    list_filter = ('modify_dt')
    # 검색박스 지정
    search_fields = ('title', 'content')
    # slug는 title 필드로 채워지도록 지정
    prepopulated_fields = {'slug' : ('title',)}
```

- 데이터베이스 변경사항 반영

```
python manage.py makemigrations blog

python manage.py migrate
```

- 테이블 생성 확인   
: http://localhost:8000/admin

## 4. URLconf 코딩하기

- 전체 URL 수정   
: mysite/urls.py에서 이름만 주고, 각 앱의 urls.py에서 자세한 url 지정

```
mysite/urls.py

from django.contrib import admin
from django.urls import path, include
# from bookmark.views import BookmarkLV, BookmarkDV
#from django.views.generic import ListView, DetailView
#from bookmark.models import Bookmark

urlpatterns = [
    path('admin/', admin.site.urls),
    path('bookmark/', include('bookmark.urls')),
    path('blog/', include('blog.urls')),
    
    #path('bookmark/', BookmarkLV.as_view(), name = 'index'),
    #path('bookmark/<int:pk>', BookmarkDV.as_view(), name = 'detail'),
    #path('bookmark/', ListView.as_view(model = Bookmark), name = 'index'),
    #path('bookmark/<int:pk>', DetailView.as_view(model = Bookmark), name = 'detail'),
]
```

- Bookmark 앱의 url   

```
from django.urls import path
from bookmark.views import BookmarkLV, BookmarkDV

app_name = 'bookmark'

url_patterns = [
    path('', BookmarkLV.as_view(), name = 'index'),
    path('<int:pk>/', BookmarkDV.as_view(), name = 'detail'),
]
```

- Blog 앱의 url

```
from django.urls import path, re_path
from blog import views

app_name = 'blog'

url_patterns = [
    # 2가지 요청을 처리하는 PostLV
    path('', views.PostLV.as_view(), name = 'index'),
    path('post/', views.PostLV.as_view(), name = 'post_list'),
    path('post/django-example/', PostDV.as_view(), name = 'post_detail'),
    path('archive/', views.PostAV.as_view(), name = 'post_archive'),
    path('archive/<int:year>/', views.PostYAV.as_view(), name = 'post_year_archive'),
    path('archive/<int:year>/<str:month>/', views.PostMAV.as_view(), name = 'post_month_archive'),
    path('archive/<int:year>/<str:month>/<int:day>/', views.PostDAV.as_view(), name = 'post_day_archive'),
    path('archive/today/', views.PostTAV.as_view(), name = 'post_today_archive')
]
```

## 5. 뷰  코딩하기

```
blog/views.py

from django.views.generic import ListView, DetailView
from django.views.generic.dates import ArchiveIndexView, YearArchiveView, MonthArchiveView
from django.views.generic.dates import DayArchiveView, TodayArchiveView
from blog.models import Post

class PostLV(ListView) :
    model = Post
    template_name = 'post_all.html'
    # 템플릿 파일로 넘아가는 객체명 지정
    context_object_name = 'posts'
    # 페이징 기능을 이용하여 한 페이지에 2개씩의 포스트만 보여줌
    paginate_by = 2
    
    
class PostDV(DetailView) :
    model = Post


class PostAV(ArchiveIndexView) :
    model = Post
    date_field = 'modify_dt'
    
    
class PostYAV(YearArchiveView) :
    model = Post
    date_field = 'modify_dt'
    
    
class PostMAV(MonthArchiveView) :
    model = Post
    date_field = 'modify_dt'
    # 년도에 해당하는 객체 리스트 만들어서 넘기기
    make_object_list = True
    # 디폴트 값 : month_format = '%b'
    
    
class PostDAV(DayArchiveView) :
    model = Post
    date_field = 'modify_dt'
    
    
class PostTAV(TodayArchiveView) :
    model = Post
    date_field = 'modify_dt'
```

## 6. 템플릿 코딩하기

- post_all.html

```
blog/templates/post_all.html

<!DOCTYPE html>
<html>

<body>
    <h1> Blog List </h1>
    <br>

    {% for post in posts %}
        # 제목을 누르면 다음 URL로 넘어감
        # get_absolute_url : /blog/post/slug단어/
        <h3><a href = '{{ post.get_absoulte_url }}'>{{ post.title }}</a></h3>
        {{ post.modify_dt | date : "N d, Y" }}
        <p> {{post.description }} </p>
    {% endfor %}

    <br>

    # 페이지 넘기는 기능
    <div>
        <span>
            # 만약 이전 페이지를 가졌다면
            {% if page_obj.has_previous %}j
                # 이전 페이지로 넘어가기 기능
                <a href = "?page={{ page_obj.previous_page_number }}">PreviousPage</a>
            {% endif %}

            # 본래 페이지
            Page {{ page_obj.number }} of {{ page_obj.paginator.num_pages }}

            # 만약 다음 페이지가 존재한다면
            {% if page_obj.has_next %}
                # 다음 페이지로 넘어가기 기능
                <a href = "?page={{ page_obj.nex_page_number }}">NextPage</a>
            {% endif %}
        </span>
    </div>

</body>
</html>
```

- post_detail.html

```
blog/templates/post_detail.html

<!DOCTYPE html>
<html>

<body>
    # 포스트 제목 보여주기
    <h2>{{ object.title }}</h2>

    <p>
        # 이전 포스트 가져오기
        {% if object.get_previous %}
        <a href = "{{ object.get_previous.get_absolute_url }}" title = "View previous post">
            &laquo;-- {{ object.get_previous }}
        </a>
        {% endif %}

        # 다음 포스트 가져오기
        {% if object.get_next %}
        | <a href = "{{ objectt.get_nex.get_absoulte.url }}" title = "View next post">
        {{ object.get_next }} --&raquo;
        </a>
        {% endif %}
    </p>

    # 포스트 수정날짜 보여주기
    <p> {{ object.modify_dt | date : "j F Y" }} </p>
    <br>

    # 포스트 내용 보여주기
    <div>
        {{ object.content | linebreaks ||
    </div>
</body>
</html>
```

- post_archive.html

```
blog/templates/post_archive.html

<!DOCTYPE html>

<html>
<body>
    <h1> Post Archives until {% now "N d, Y" %}</h1>
    
    # 순서 없는 리스트
    <ul>
        # 포스트가 작성된 연도 모두 보여주고 해당 년도로 이동하는 url
        {% for date in date_list %}
            <li style = "display : inline;">
                <a href = "{% url 'blog:post_year_archive' date| date 'Y' %}">Year {{ date | date : "Y" }}</a>
            </li>
        {% endfor %}
    </ul>

    <br>
    
    # 모든 포스트 리스트 보여주기
    <div>
        <ul>
            {% for post in object_list %}
            <li>{{ post.modify_dt | date : "Y-m-d" }} &nbsp;&nbsp;&nbsp;
            <a href = "{{ post.get_absolute_url }}"><strong>{{ post.title }}</strong></a>
            </li>
            {% endfor %}
        </ul>
    </div>

</body>
</html>
```

- post_archive_year.html

```
<!DOCTYPE html>

<html>
<body>
    # 해당 년도의 포스트들
    <h1>Post Archives for {{ year | date : "Y" }}</h1>

    # 순서 없는 리스트
    <ul>
        # 해당 년도의 존재하는 달 모두 보여주기
        {% for date in date_list %}
        <li style = "display : inline;">
            <a href = "{% url 'blog:post_month_archive' year | date : 'Y' date | date : 'b' %}"> {{ date | date : "F" }}</a>
        </li>
        {% endfor %}
    </ul>

    <br>

    <div>
        # 순서없는 리스트
        <ul>
            # 해당 년도의 모든 제목 보여주기
            {% for post in object_list %}
            <li>{{ post.modify_dt | date " "Y-m-d" }} &nbsp;&nbsp;&nbsp;
            <a href = "{{ post.get_absoulte_url }}"><strong>{{ post.title }}</strong></a>
            </li>
            {% endfor %}
        </ul>
    </div>

</body>
</html>
```

- post_archive_month.html

```
<!DOCTYPE html>

<html>
<body>
    <h1> Posts Archives for {{ month | date : "N, Y" }}</h1>

    <div>
        <ul>
            {% for post in object_list %}
            <li>{{ post.modify_dt | datE : "Y-m-d" }} &nbsp;&nbsp;&nbsp;
            <a href = "{{ post.get_absoulte_url }}"><strong>{{ post.title }}</strong></a>
            </li>
            {% endfor %}
        </ul>
    </div>

</body>
</html>
```

- post_archive_day.html

```
<!DOCTYPE html>

<html>
<body>
    <h1>Posts Archive for {{ day | date : "N d, Y" }}</h1>

    <div>
        <ul>
            {% for post in object_list %}
            <li>{{ post.modify_dt | date : "Y-m-d" }} &nbsp;&nbsp;&nbsp;
            <a href = "{{ post.get_absolute_url }}"><strong>{{ post.title }}</strong></a>
            </li>
            {% endfor %}
        </ul>
    </div>

</body>
</html>
```

## 확인하기

http://localhost:8000/blog